In [1]:
import tensorflow as tf 
import pandas as pd 
import numpy as np 

In [2]:
from tensorflow import keras
from keras import layers as tfl
def convolutional_model(input_shape):
    input_img = tf.keras.Input(shape=input_shape)
    conv_layer1 = tfl.Conv2D(filters = 16 , kernel_size= 3 , strides= 1 , padding='same')(input_img)
    activation1 = tfl.ReLU()(conv_layer1)
    pool_layer1 = tfl.MaxPool2D(pool_size=8, strides=8, padding='same')(activation1)
    conv_layer2 = tfl.Conv2D(filters= 64 , kernel_size= 2 , strides= 1 , padding='same')(pool_layer1)
    activation2 = tfl.ReLU()(conv_layer2)
    pool_layer2 = tfl.MaxPool2D(pool_size=4, strides=4, padding='same')(activation2)
    flatten =  tfl.Flatten()(pool_layer2)
    outputs =  tfl.Dense(6, activation='softmax')(flatten)
    model = tf.keras.Model(inputs=input_img, outputs=outputs)
    return model

conv_model = convolutional_model((128, 128, 1))
conv_model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
conv_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 128, 128, 1)]     0         
                                                                 
 conv2d (Conv2D)             (None, 128, 128, 16)      160       
                                                                 
 re_lu (ReLU)                (None, 128, 128, 16)      0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 16, 16, 16)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 16, 16, 64)        4160      
                                                                 
 re_lu_1 (ReLU)              (None, 16, 16, 64)        0         
                                                             

In [4]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rotation_range=40,rescale=1./255,width_shift_range=0.2,
        height_shift_range=0.2,shear_range=0.2,
        zoom_range=0.4,brightness_range=[0.4,1.5])
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rotation_range=40,rescale=1./255,width_shift_range=0.2,
        height_shift_range=0.2,shear_range=0.2,
        zoom_range=0.4,brightness_range=[0.4,1.5])


In [5]:
train_dir_new = "C:/Users/91898/Desktop/Finger Counting/Finger_Classes"
test_dir_new = "C:/Users/91898/Desktop/Finger Counting/Finger_Classes_test"

In [6]:
train_generator = train_datagen.flow_from_directory(train_dir_new,
                                                   target_size = (128,128), batch_size = 32 , class_mode='categorical',color_mode='grayscale')
test_generator = test_datagen.flow_from_directory(test_dir_new,
                                                   target_size = (128,128), batch_size = 32 , class_mode='categorical',color_mode='grayscale')

Found 18085 images belonging to 6 classes.
Found 3600 images belonging to 6 classes.


In [7]:
model = conv_model.fit_generator(train_generator,steps_per_epoch=200,epochs=75,shuffle=True)


<ipython-input-7-1486996ba1cb>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model = conv_model.fit_generator(train_generator,steps_per_epoch=200,epochs=75,shuffle=True)


Epoch 1/75
200/200 [==============================] - 22s 106ms/step - loss: 1.7408 - accuracy: 0.2430
Epoch 2/75
200/200 [==============================] - 19s 93ms/step - loss: 1.5171 - accuracy: 0.3841
Epoch 3/75
200/200 [==============================] - 18s 88ms/step - loss: 1.2792 - accuracy: 0.4934
Epoch 4/75
200/200 [==============================] - 17s 86ms/step - loss: 1.1509 - accuracy: 0.5480
Epoch 5/75
200/200 [==============================] - 18s 89ms/step - loss: 1.0530 - accuracy: 0.5859
Epoch 6/75
200/200 [==============================] - 16s 82ms/step - loss: 1.0110 - accuracy: 0.5958
Epoch 7/75
200/200 [==============================] - 16s 80ms/step - loss: 0.9544 - accuracy: 0.6239
Epoch 8/75
200/200 [==============================] - 16s 82ms/step - loss: 0.9231 - accuracy: 0.6381
Epoch 9/75
200/200 [==============================] - 17s 84ms/step - loss: 0.9046 - accuracy: 0.6628
Epoch 10/75
200/200 [==============================] - 17s 86ms/step - loss: 0.83

In [8]:
def resize(img_address):
    import cv2 
    img = cv2.imread(img_address)
    image = cv2.resize(img,(128,128))
    return image

def grayscale(image_address):
    from skimage import io,color
    
    image = color.rgb2gray(image_address)
    io.imshow(image)
        
    
    return image

def gauss(img):
    mean = 0
    var = 0.1
    sigma = var ** 0.5
    gaussian = np.random.normal(mean,var,(128, 128)) #  np.zeros((224, 224), np.float32)
    pareto = np.random.pareto(40,(128,128))
    poisson = np.random.poisson(0.003,(128,128))
    
    noisy_image = np.zeros(img.shape, np.float32)

    if len(img.shape) == 2:
        noisy_image = img+pareto
    else:
        noisy_image[:, :, 0] = img[:, :, 0] + gaussian
        noisy_image[:, :, 1] = img[:, :, 1] + gaussian
        noisy_image[:, :, 2] = img[:, :, 2] + gaussian

    cv2.normalize(noisy_image, noisy_image, 0, 255, cv2.NORM_MINMAX, dtype=-1)
    noisy_image = noisy_image.astype(np.uint8)
    
    plt.imshow(noisy_image,cmap='gray')
    
    return noisy_image

In [ ]:
image_address=r"C:/Users/91898/Downloads/Showing_1.png"
img = resize(image_address)

image = grayscale(img)
#image = gauss(image)
image = np.expand_dims(image,0)
len(image.shape)

In [32]:
pred = conv_model.predict(image)

1/1 [==============================] - 0s 16ms/step


In [33]:
def give_user_pred(arr,var=0):
    for a in range(len(arr[0])):
        if arr[0][a]>var:
            var = arr[0][a]
            index = a 
    return index 

In [34]:
print(give_user_pred(pred))  #FINAL PREDICTION

1
